In [309]:
CLIENT_ID = "doZap8T-60t8-8SsgzKPxg"
# Add secret key
SECRET_KEY  = "MTfNpx9VzIj1ZB6vk6NSDnYeEtpjxA"

In [310]:
with open('pwd.txt', 'r') as file:
    pw = file.read()

# Get Reddit Data

## Getting Access To Reddit API

In [311]:
import praw
reddit = praw.Reddit(client_id = CLIENT_ID,
                     client_secret = SECRET_KEY,
                     username = 'Carlosshadow5',
                     password = pw,
                     user_agent = 'NPL_Project')

## Getting Data by Search

In [312]:
import pandas as pd

# Gets the subreddit
subreddit = reddit.subreddit("BigFive")

### Neuroticism Posts

In [313]:
neuroticism_data = {}
neuroticism_df = pd.DataFrame(neuroticism_data)
neuroticism_user_names = []

subreddit_search = subreddit.search("neuroticism", limit = None)
for post in subreddit_search:
    neuroticism_user_names.append(post.author.name)

# Get unique only
neuroticism_user_names = list(set(neuroticism_user_names))

neuroticism_df['User Name'] = neuroticism_user_names
neuroticism_df

,User Name
0,Zestyclose-Tower3859
1,Proud_Echidna_4960
2,Aegeblomme_MinouKane
3,Reghretti
4,RoseTheQuartz37
...,...
169,lR5Yl
170,Cognitively_Absurd
171,_un1ty
172,killerlumberjack


### Extraversion Posts

In [314]:
extraversion_data = {}
extraversion_df = pd.DataFrame(extraversion_data)
extraversion_user_names = []

subreddit_search = subreddit.search("extraversion", limit = None)
for post in subreddit_search:
    extraversion_user_names.append(post.author.name)

# Get unique only
extraversion_user_names = list(set(extraversion_user_names))

extraversion_df['User Name'] = extraversion_user_names
extraversion_df

,User Name
0,Proud_Echidna_4960
1,raisinindasun
2,Logical_Row3506
3,Aegeblomme_MinouKane
4,abbyrosalina
...,...
119,PalpitationSlight751
120,Cognitively_Absurd
121,_un1ty
122,killerlumberjack


### Openness Posts

In [315]:
openness_data = {}
openness_df = pd.DataFrame(openness_data)
openness_user_names = []

subreddit_search = subreddit.search("openness", limit = None)
for post in subreddit_search:
    openness_user_names.append(post.author.name)

# Get unique only
openness_user_names = list(set(openness_user_names)) 

openness_df['User Name'] = openness_user_names
openness_df

,User Name
0,Proud_Echidna_4960
1,raisinindasun
2,American-_-Nightmare
3,Aegeblomme_MinouKane
4,Reghretti
...,...
172,PalpitationSlight751
173,Cognitively_Absurd
174,_un1ty
175,killerlumberjack


### Agreeableness Posts

In [316]:
agreeableness_data = {}
agreeableness_df = pd.DataFrame(agreeableness_data)
agreeableness_user_names = []

subreddit_search = subreddit.search("agreeableness", limit = None)
for post in subreddit_search:
    agreeableness_user_names.append(post.author.name)

# Get unique only
agreeableness_user_names = list(set(agreeableness_user_names)) 

agreeableness_df['User Name'] = agreeableness_user_names
agreeableness_df

,User Name
0,Zestyclose-Tower3859
1,nationofcake
2,Proud_Echidna_4960
3,Logical_Row3506
4,Aegeblomme_MinouKane
...,...
166,Cognitively_Absurd
167,luckis4losersz
168,_un1ty
169,killerlumberjack


### Conscientiousness Posts

In [317]:
conscientiousness_data = {}
conscientiousness_df = pd.DataFrame(conscientiousness_data)
conscientiousness_user_names = []

subreddit_search = subreddit.search("conscientiousness", limit = None)
for post in subreddit_search:
    conscientiousness_user_names.append(post.author.name)

# Get unique only
conscientiousness_user_names = list(set(conscientiousness_user_names)) 

conscientiousness_df['User Name'] = conscientiousness_user_names
conscientiousness_df

,User Name
0,Proud_Echidna_4960
1,raisinindasun
2,Emma_Oakley
3,abbyrosalina
4,Luster-Star
...,...
160,lR5Yl
161,Cognitively_Absurd
162,_un1ty
163,killerlumberjack


# Format Data

## Create User Class

In [318]:
import json
from json import JSONEncoder
from enum import Enum
from typing import Any

class Personality(Enum):
    EXTRAVERSION = 1,
    NEUROTICISM = 2,
    OPENNESS = 3,
    AGREEABLENESS = 4,
    CONSCIENTIOUSNESS = 5

class User:
    def __init__(self, data_string, personality_type):
        self.data = data_string
        self.personality = personality_type
    def __str__(self):
        return str(self.personality) + "\n " +  str(self.data)
    
# Define a custom encoder for the class and enum
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, User):
            return {'__class__': 'User',
                    'data': obj.data,
                    'personality': obj.personality.name}
        if isinstance(obj, Enum):
            return obj.name
        return json.JSONEncoder.default(self, obj)


## Get User String Data

In [319]:
def getUserStringData(username):
    string_data = ""
    user = reddit.redditor(username)
    
    posts = list(user.submissions.new(limit = None))
    comments = list(user.comments.new(limit=None))
    if len(comments) < 5 and len(posts) < 5:
        return ""
    
    # filter posts
    for post in posts:
        string_data += post.title
        string_data += "\n" + post.selftext

    # filter comments
    for comment in comments:
        string_data += str(comment.body)
        string_data += "\n\n"
    string_data = string_data.replace("\n\n&#x200B;\n\n", " \n ")
    string_data = string_data.replace("&#x200B;", " \n")
    
    return string_data


In [320]:
from prawcore.exceptions import Forbidden

users = {}

# Add users from any group
def addUserDict(users_list, personality_type):
    for username in users_list:
        try:
            user_string_data = getUserStringData(username)
            if user_string_data == "":
                users_list.remove(username)
                continue
            user_data_object = User(user_string_data, personality_type)
            users[str(username) + f"{personality_type}"[11:].lower()] = user_data_object
        except Forbidden:
            print("ERROR: USER/COMMENT NOT FOUND\n\n")
          
# test = ["Zestyclose-Tower3859", "Proud_Echidna_4960", "nationofcake", "Emma_Oakley", "Reghretti"]
# addUserDict(test, Personality.OPENNESS)

In [321]:
addUserDict(neuroticism_user_names, Personality.NEUROTICISM)
addUserDict(extraversion_user_names, Personality.EXTRAVERSION)
addUserDict(openness_user_names, Personality.OPENNESS)
addUserDict(agreeableness_user_names, Personality.AGREEABLENESS)
addUserDict(conscientiousness_user_names, Personality.CONSCIENTIOUSNESS)


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: USER/COMMENT NOT FOUND


ERROR: U

# Save Data

In [322]:
# Save the dictionary to a file
with open('users.json', 'w') as f:
    json.dump(users, f, cls=MyEncoder, indent=4)

# Load Data from Json File

In [336]:
import json

data = ""
# Load the dictionary from the file
with open('users.json', 'r') as f:
    data = json.load(f)

# Convert the dictionary back to objects
def my_object_hook(dct):
    if '__class__' in dct:
        if dct['__class__'] == 'MyClass':
            return User(dct['data'], Personality[dct['personality']])
    return dct

# Convert the dictionary back to objects
users = json.loads(json.dumps(data), object_hook=my_object_hook)

# Print the loaded dictionary
print(len(users))
print(type(users))

643
<class 'dict'>


When the json file is read to retrieve "users", the User class is read as a dict and not an object. 

This means the way to access its data is to use these keys: '__class__', 'data', 'personality'.

The '__class__' key returns User (the name of the class). The 'data' returns the data string. The 'personality' key returns the personality as a string. 

In [337]:
users["Zestyclose-Tower3859.neuroticism"]

{'__class__': 'User',
 'data': 'Solving Pelvic Floor problems with stretches, lifestyle changes, etc?\nHas anybody here actually solved their pelvic floor problems and symptoms (constipation, pain, etc) by changes in diet, lifestyle, and using pelvic floor stretches? \n\nBiofeedback therapy is not within my reach for now, so I was hoping to do what I can to help alleviate my problem. Any insights would be highly appreciated.Going to a pelvic floor therapist?\nHi there. \n\nI\'m planning to go to a pelvic floor therapist because of constipation symptoms that point out to dyssynergia. How do they diagnose if you have dyssynergic defecation? Does it require anorectal manometry? I\'m asking because anorectal manometry is extremely expensive here in my country so I\'m hoping they just stick a finger or something instead.Solfigreen users here?\nHi there, \n\nMy motility expert doctor recommended to me a supplement of sorts that came from Japan. It\'s just all natural, lots of fiber in it. I 

# Making the Models

### Preprocess data

In [338]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from enum import Enum

In [339]:
# get all user objects since usernames are useless
user_objects = [users[username] for username in users.keys()]
personalities = [user["personality"] for user in user_objects]

## Positive/Negative Score Model

In [340]:
import pandas as pd

df = pd.read_excel('../data/word_list_pos_neg.xlsx')
positive_words = df['Positive Sense Word List'].dropna()[1:]
negative_words = df['Negative Sense Word List'][1:]
all_words = (positive_words + negative_words).drop_duplicates()

In [341]:
def calculate_connotation_score(input_string):
    # Calculate the count vector for the input string
    vectorizer = TfidfVectorizer(vocabulary = positive_words, lowercase=True)
    positive_counts = vectorizer.fit_transform([input_string])
    positive_counts = positive_counts.toarray()[0]

    vectorizer = TfidfVectorizer(vocabulary = negative_words, lowercase=True)
    negative_counts = vectorizer.fit_transform([input_string])
    negative_counts = negative_counts.toarray()[0]

    positive_count = sum(positive_counts)
    negative_count = sum(negative_counts)

    connotation_score = positive_count - negative_count

    return round(connotation_score, 3)

In [342]:
import numpy as np

vectorizer = TfidfVectorizer(dtype=np.float64)
all_strings = [user["data"] for user in user_objects]
scored_strings = [str(calculate_connotation_score(string)) for string in all_strings]

strings_vector = vectorizer.fit_transform(scored_strings)

In [343]:
# Split data into training and testing sets
splitter = StratifiedShuffleSplit(n_splits=10, train_size=0.80, test_size=0.20, random_state=9)
train_indices, test_indices = next(splitter.split(strings_vector, personalities))
train_data = strings_vector[train_indices]
train_personalities = [personalities[i] for i in train_indices]
test_data = strings_vector[test_indices]
test_personalities = [personalities[i] for i in test_indices]

In [344]:
# Train the Gaussian Naive Bayes classifier
clf = GaussianNB()
clf.fit(train_data.toarray(), train_personalities)

predicted_personalities = clf.predict(test_data.toarray())
print(classification_report(test_personalities, predicted_personalities))

                   precision    recall  f1-score   support

    AGREEABLENESS       0.00      0.00      0.00        28
CONSCIENTIOUSNESS       0.00      0.00      0.00        26
     EXTRAVERSION       0.03      0.11      0.04        19
      NEUROTICISM       0.00      0.00      0.00        28
         OPENNESS       0.00      0.00      0.00        28

         accuracy                           0.02       129
        macro avg       0.01      0.02      0.01       129
     weighted avg       0.00      0.02      0.01       129



## Word Count Model

In [345]:
vectorizer = TfidfVectorizer(stop_words="english") 
string_data = vectorizer.fit_transform(user["data"] for user in user_objects)

In [346]:
# Split data into training and testing sets
splitter = StratifiedShuffleSplit(n_splits=10, train_size=0.80, test_size=0.20, random_state=9)
train_indices, test_indices = next(splitter.split(string_data, personalities))
train_data = string_data[train_indices]
train_personalities = [personalities[i] for i in train_indices]
test_data = string_data[test_indices]
test_personalities = [personalities[i] for i in test_indices]

In [347]:
# Train the Gaussian Naive Bayes classifier
clf = GaussianNB()
clf.fit(train_data.toarray(), train_personalities)

predicted_personalities = clf.predict(test_data.toarray())
print(classification_report(test_personalities, predicted_personalities))

                   precision    recall  f1-score   support

    AGREEABLENESS       0.12      0.04      0.06        28
CONSCIENTIOUSNESS       0.13      0.08      0.10        26
     EXTRAVERSION       0.00      0.00      0.00        19
      NEUROTICISM       0.06      0.07      0.06        28
         OPENNESS       0.07      0.04      0.05        28

         accuracy                           0.05       129
        macro avg       0.08      0.04      0.05       129
     weighted avg       0.08      0.05      0.06       129



## Sentiment Analysis Model

In [348]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

sentiment_scores = []
for i in range(len(user_objects)):
    sentiment_scores.append(analyzer.polarity_scores(user_objects[i]['data']))

# Add sentiment scores as additional features to the input data
full_data = np.array([[score['pos'], score['neg'], score['neu']] for score in sentiment_scores])

In [349]:
# Split data into training and testing sets
splitter = StratifiedShuffleSplit(n_splits=10, train_size=0.80, test_size=0.20, random_state=9)
train_indices, test_indices = next(splitter.split(full_data, personalities))
train_data = full_data[train_indices]
train_personalities = [personalities[i] for i in train_indices]
test_data = full_data[test_indices]
test_personalities = [personalities[i] for i in test_indices]


In [350]:
# Train the Gaussian Naive Bayes classifier
clf = GaussianNB()
clf.fit(train_data, train_personalities)

# Test the classifier and print the classification report
predicted_personalities = clf.predict(test_data)
print(classification_report(test_personalities, predicted_personalities))

                   precision    recall  f1-score   support

    AGREEABLENESS       0.21      0.11      0.14        28
CONSCIENTIOUSNESS       0.00      0.00      0.00        26
     EXTRAVERSION       0.00      0.00      0.00        19
      NEUROTICISM       0.16      0.43      0.24        28
         OPENNESS       0.31      0.32      0.32        28

         accuracy                           0.19       129
        macro avg       0.14      0.17      0.14       129
     weighted avg       0.15      0.19      0.15       129



In [351]:
# print(test_personalities[:10])
# print(predicted_personalities[:10])

for x in range(20):
    print(test_personalities[x] + "  -  " + predicted_personalities[x])

AGREEABLENESS  -  AGREEABLENESS
AGREEABLENESS  -  NEUROTICISM
AGREEABLENESS  -  NEUROTICISM
AGREEABLENESS  -  NEUROTICISM
OPENNESS  -  OPENNESS
AGREEABLENESS  -  NEUROTICISM
OPENNESS  -  OPENNESS
OPENNESS  -  NEUROTICISM
NEUROTICISM  -  NEUROTICISM
EXTRAVERSION  -  NEUROTICISM
AGREEABLENESS  -  NEUROTICISM
CONSCIENTIOUSNESS  -  OPENNESS
NEUROTICISM  -  AGREEABLENESS
OPENNESS  -  NEUROTICISM
CONSCIENTIOUSNESS  -  OPENNESS
NEUROTICISM  -  CONSCIENTIOUSNESS
OPENNESS  -  OPENNESS
CONSCIENTIOUSNESS  -  NEUROTICISM
NEUROTICISM  -  EXTRAVERSION
AGREEABLENESS  -  OPENNESS
